## Setting
### Mounting google drive

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

### Download Mecab
Dacon에서 공개한(?) Window Mecab version

In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab

/content/gdrive/My Drive/Colab Notebooks/smishing/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

### Import packages

In [0]:
# data preprocessing
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
# tokenizer
import re
from konlpy.tag import Mecab
# modeling
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM, Bidirectional, Flatten, Input, SpatialDropout1D, Conv1D, MaxPooling1D, GRU, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
import pickle
import warnings
warnings.filterwarnings(action='ignore') 

### Load Data

In [0]:
path = '/content/gdrive/My Drive/Colab Notebooks/smishing/'
train = pd.read_csv(path+'data/train.csv')
test = pd.read_csv(path+'data/public_test.csv')
submission=pd.read_csv(path+'data/submission_제출양식.csv')
print(train.shape, test.shape, submission.shape)

## Data Preparation
### Sampling
- Counter({0: 277242, 1: 18703}) #0.0631975535994864
- mixed sampling

In [0]:
RANDOM_SEED = 1234

## for train data
# index
train_nsm_idx=list(train[train['smishing']==0].index)
train_sm_idx=list(train[train['smishing']==1].index)
# 3. Mixed Sampling
random.seed(RANDOM_SEED)
train_nsm_idx = random.sample(train_nsm_idx, k=18703*6)
random.seed(RANDOM_SEED)
train_sm_idx = random.choices(train_sm_idx, k=18703*3)
# index shuffle
train_idx = train_nsm_idx + train_sm_idx
print(train_idx[:5])
random.shuffle(train_idx)
print(train_idx[:5])
# Merge
train_X = pd.DataFrame(train['text'], columns=['text']).iloc[train_idx].reset_index(drop=True)
train_y = pd.DataFrame(train['smishing'], columns=['smishing']).iloc[train_idx].reset_index(drop=True)
print(train_X.shape, train_y.shape)

[242186, 63505, 4255, 49550, 19561]
[96380, 19368, 145827, 73965, 10314]
(168327, 1) (168327, 1)


In [0]:
## for test data
test['smishing'] = 2 # train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_X = pd.DataFrame(test['text'])
test_y = pd.DataFrame(test['smishing'])
print(test_X.shape, test_y.shape)

(1626, 1) (1626, 1)


### Tokenizing

In [0]:
tmp = [len(x) for x in train_X.text]
pd.DataFrame(tmp).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
0,168327.0,355.887047,357.894367,2.0,42.0,171.0,739.0,1478.0


In [0]:
stopwords = ['을', '를', '이', '가', '은', '는', 'null']

def tokenizing(text_list):
  tokenizer = Mecab()
  token_list = []
  for text in text_list:
    tokens = []
    txt = re.sub('[^가-힣a-zA-Z]',' ',text)
    txt = re.sub('X{1,}',' ',txt)
    token = tokenizer.morphs(txt)
    for t in token:
      if t not in stopwords:
        tokens.append(t)
    token_list.append(' '.join(tokens))
  return token_list

In [0]:
train_X_txt = tokenizing(train_X.text)
test_X_txt = tokenizing(test_X.text)

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_X_txt)  # _개의 행을 가진 X의 각 행에 토큰화를 수행
train_X_seq = tokenizer.texts_to_sequences(train_X_txt) #단어를 숫자값, 인덱스로 변환하여 저장
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1
print('vocab size: ', vocab_size)
max_len = 1000 # 전체 데이터셋의 길이 맞추기
X_train = pad_sequences(train_X_seq, maxlen=max_len)
print("train data shape: ", X_train.shape)
test_X_seq = tokenizer.texts_to_sequences(test_X_txt) #단어를 숫자값, 인덱스로 변환하여 저장
X_test = pad_sequences(test_X_seq, maxlen=max_len)
print("test data shape: ", X_test.shape)

vocab size:  28084
train data shape:  (168327, 1000)
test data shape:  (1626, 1000)


## Modeling

In [0]:
auc_score_=1
def auc_score(y_true, y_pred):
  global auc_score_
  try:
    auc_score_ = roc_auc_score( y_true, y_pred, average='macro', sample_weight=None).astype('float32')
  except ValueError:
    pass
  return auc_score_

def auc( y_true, y_pred ) :
    score = tf.py_func( lambda y_true, y_pred : auc_score(y_true, y_pred) , [y_true, y_pred], 'float32', stateful=False, name='sklearnAUC' )
    return score

### Conv Bi GRU
##### (1) train

In [0]:
model= Sequential()
model.add(Embedding(vocab_size,128,input_length=max_len))
model.add(Conv1D(64,kernel_size=3,activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(128,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=[auc])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 128)         3594752   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 998, 64)           24640     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 499, 64)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 499, 64)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 499, 256)          148224    
_________________________________________________________________
dropout_7 (Dropout)          (None, 499, 256)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 127744)           

In [0]:
history = model.fit(X_train, train_y, epochs=3, batch_size=128, validation_split=0.2)
model_json = model.to_json()
with open(path+'model/'+'ConvBiGRU_v3.json', 'w') as json_file : 
    json_file.write(model_json)

Train on 134661 samples, validate on 33666 samples
Epoch 1/3
134661/134661 [==============================] - 2147s 16ms/sample - loss: 0.0126 - auc: 0.9988 - val_loss: 0.0014 - val_auc: 1.0000
Epoch 2/3
134661/134661 [==============================] - 2139s 16ms/sample - loss: 0.0012 - auc: 1.0000 - val_loss: 6.8427e-04 - val_auc: 1.0000
Epoch 3/3
 36736/134661 [=======>......................] - ETA: 25:36 - loss: 5.2580e-04 - auc: 1.0000

In [0]:
# 귀찮
y_pred = model.predict(X_test, batch_size=128)
print(y_pred[:4])
submission['smishing'] = y_pred
submission.to_csv(path+'submission/'+'ConvBiGRU_v3.csv',index=False)

In [0]:
 v submission.iloc[np.where(submission.smishing>0.8)]

,id,smishing
192,340192,0.999518
259,340259,1.000000
261,340261,1.000000
267,340267,1.000000
268,340268,1.000000
...,...,...
1609,341609,1.000000
1610,341610,1.000000
1611,341611,1.000000
1612,341612,1.000000


##### (2) validation

In [0]:
scores = model.evaluate(X_test, y_test, verbose=0) # 테스트 데이터에 대해서 정확도 평가
print("정확도: %.2f%%" % (scores[1]*100))

In [0]:
epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

##### (3) find the best

In [0]:
y_pred = model.predict(X_test, batch_size=64)
print(y_pred.shape)

### Submission
test data에서 각 text당 스미싱 확률을 구할 수 있으니 이를 제출양식의 smishing 변수에 넣고 csv 파일로 내보낸다

In [0]:
submission['smishing'] = y_pred
submission.to_csv(path+'submission/'+"LSTM_1.csv",index=False)

In [0]:
s = pd.read_csv(spath+'LSTM_1.csv')
s.head()

,id,smishing
0,340000,0.0
1,340001,0.0
2,340002,0.0
3,340003,0.0
4,340004,0.0
